
**Install requirements**

In [0]:
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'

  Using cached https://files.pythonhosted.org/packages/88/95/90e8c4c31cfc67248bf944ba42029295b77159982f532c5689bcfe4e9108/torch-1.3.1-cp36-cp36m-manylinux1_x86_64.whl
ERROR: torchvision 0.5.0 has requirement torch==1.4.0, but you'll have torch 1.3.1 which is incompatible.
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0
  Using cached https://files.pythonhosted.org/packages/24/19/4804aea17cd136f1705a5e98a00618cb8f6ccc375ad8bfa437408e09d058/torch-1.4.0-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1


**Import libraries**

In [0]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader, ConcatDataset
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet
from torchvision.models import vgg16

from PIL import Image
from tqdm import tqdm

from torchvision.datasets import VisionDataset
import os.path
import sys

from sklearn.model_selection import train_test_split
import numpy as np
import copy

import matplotlib.pyplot as plt

**Set Arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 101 # 101 + 1: There is am extra Background class that should be removed 

# DEFAULT VALUES

#BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 0.001            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

#NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

# SECOND CONFIGURATION
#LR = 0.01           
#NUM_EPOCHS = 40
#GAMMA = 0.1

# THIRD CONFIGURATION
#LR = 0.01           
#NUM_EPOCHS = 30
#GAMMA = 0.2

# FOURTH CONFIGURATION
#LR = 0.01           
#NUM_EPOCHS = 30
#GAMMA = 0.8

# THIRD CONFIGURATION - Transfer Learning
#LR = 0.01           
#NUM_EPOCHS = 30
#STEP_SIZE = 10
#GAMMA = 0.8

transferLearning = True #True if we use Transfer Learning, False otherwise

train_freeze = "all" 
# "all" train all layers
# "fc" train only fully connected layers and freeze other layers
# "cl" train only convolutional layers and freeze other layers

#### VGGNet ####

network = "vgg"
# "alex" if AlexNet is the network that we want to use
# "vgg" if VGGs is the network that we want to use

BATCH_SIZE = 16
NUM_EPOCHS = 10

**Define Data Preprocessing**

In [0]:
if transferLearning: # ImageNet
  mean = (0.485, 0.456, 0.406)
  std = (0.229, 0.224, 0.225)
else:
  mean = (0.5, 0.5, 0.5)
  std = (0.5, 0.5, 0.5)


# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize(mean, std) # Normalizes tensor with mean and standard deviation
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean, std)                                    
])


augmentation_transform1 = transforms.Compose([transforms.Resize(256),      
                                      transforms.RandomCrop(224),                      
                                      transforms.ToTensor(), 
                                      transforms.Normalize(mean, std)
])

augmentation_transform2 = transforms.Compose([transforms.Resize(256),      
                                      transforms.RandomHorizontalFlip(),
                                      transforms.CenterCrop(224),                  
                                      transforms.ToTensor(), 
                                      transforms.Normalize(mean, std)
])

augmentation_transform3 = transforms.Compose([transforms.Resize(256),      
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ColorJitter(brightness=20, contrast=10, saturation=30),
                                      transforms.CenterCrop(224),                  
                                      transforms.ToTensor(), 
                                      transforms.Normalize(mean, std)
])

**Prepare Dataset**

In [0]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

def getDataset(dir, indexClass, split="train"):
    dataset = []

    path = os.path.join(dir, split+".txt")
    with open(path, "r") as f:
        imgs = f.read().splitlines()

    for img in sorted(imgs):
        label = img.split("/")[0]

        if label != "BACKGROUND_Google":
            image = pil_loader(dir+"101_ObjectCategories/"+img)
            item = (image, indexClass[label])
            dataset.append(item)
  
    return dataset # tuple (image, index)



class Caltech(VisionDataset):
    def __init__(self, root, split='train', transform=None, target_transform=None):
        super(Caltech, self).__init__(root, transform=transform, target_transform=target_transform)

        self.split = split # This defines the split you are going to use
                           # (split files are called 'train.txt' and 'test.txt')

        '''
        - Here you should implement the logic for reading the splits files and accessing elements
        - If the RAM size allows it, it is faster to store all data in memory
        - PyTorch Dataset classes use indexes to read elements
        - You should provide a way for the __getitem__ method to access the image-label pair
          through the index
        - Labels should start from 0, so for Caltech you will have lables 0...100 (excluding the background class) 
        '''
        classes, indexClass = self._getClasses(self.root)
        dataset = getDataset(self.root, indexClass, split=self.split)

        self.classes = classes
        self.indexClass = indexClass
        self.dataset = dataset

    def _getClasses(self, dir):
        classes = os.listdir(dir+"101_ObjectCategories")
        classes.remove("BACKGROUND_Google")
        classes.sort()
        indexClass = {classes[i]: i for i in range(len(classes))}

        return classes, indexClass

    def __getitem__(self, index):
        '''
        __getitem__ should access an element through its index
        Args:
            index (int): Index
        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        '''

        image, label = self.dataset[index] # Provide a way to access image and label via index
                           # Image should be a PIL Image
                           # label can be int

        # Applies preprocessing when accessing the image
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        '''
        The __len__ method returns the length of the dataset
        It is mandatory, as this is used by several other components
        '''
        length = len(self.dataset) # Provide a way to get the length (number of elements) of the dataset
        return length

In [5]:
# Clone github repository with data
if not os.path.isdir('./Caltech101'):
  !git clone https://github.com/MachineLearning2020/Homework2-Caltech101.git
  !mv 'Homework2-Caltech101' 'Caltech101'

DATA_DIR = 'Caltech101/'

# Prepare Pytorch train/test Datasets
train_dataset = Caltech(DATA_DIR, split='train',  transform=train_transform) # we will use this dataset to not apply data augmentation to the validation set

#train_dataset_transf = Caltech(DATA_DIR, split='train',  transform=augmentation_transform1) # we will need this for data augmentation
test_dataset = Caltech(DATA_DIR, split='test', transform=eval_transform)


#train_indexes = # split the indices for your train split
#val_indexes = # split the indices for your val split

#train_dataset = Subset(train_dataset, train_indexes)
#val_dataset = Subset(train_dataset, val_indexes)

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
#print('Valid Dataset: {}'.format(len(val_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

Train Dataset: 5784
Test Dataset: 2893


In [6]:
targets = [train_dataset[i][1] for i in range(len(train_dataset))]

train_indexes, val_indexes = train_test_split(np.arange(len(targets)), test_size = 0.5, shuffle=True, stratify=targets, random_state=42)
print(sorted(train_indexes))
print(train_indexes)
print()
print(sorted(val_indexes))
print(val_indexes)

[3, 5, 6, 7, 9, 10, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 30, 31, 33, 37, 39, 42, 45, 46, 48, 54, 55, 56, 57, 59, 60, 63, 66, 67, 68, 69, 72, 73, 75, 76, 77, 78, 79, 81, 82, 84, 86, 90, 92, 93, 94, 96, 97, 99, 101, 104, 106, 108, 109, 110, 111, 113, 114, 116, 118, 119, 121, 124, 125, 126, 132, 137, 139, 140, 143, 144, 145, 146, 148, 149, 152, 153, 155, 157, 158, 163, 164, 165, 166, 167, 172, 173, 175, 176, 177, 179, 181, 182, 183, 185, 188, 191, 193, 194, 195, 199, 202, 203, 204, 208, 209, 210, 218, 219, 220, 221, 222, 225, 226, 227, 229, 231, 237, 243, 244, 248, 250, 253, 254, 257, 260, 261, 262, 266, 270, 273, 274, 275, 276, 277, 278, 279, 287, 288, 292, 296, 298, 300, 303, 306, 307, 310, 314, 316, 317, 319, 320, 321, 324, 326, 327, 329, 330, 331, 332, 333, 334, 335, 339, 345, 346, 348, 350, 353, 355, 357, 360, 361, 362, 364, 366, 367, 368, 370, 371, 372, 376, 377, 380, 384, 386, 387, 391, 394, 396, 399, 400, 401, 402, 404, 408, 410, 412, 419, 420, 421, 422, 423, 424, 425, 428,

In [7]:
print(f"Train Dataset: {len(train_indexes)}")
print(f"Valid Dataset: {len(val_indexes)}")

Train Dataset: 2892
Valid Dataset: 2892


In [0]:
val_dataset = Subset(train_dataset, val_indexes) # in order to not apply data augmentation to the validation
#train_dataset_1 = Subset(train_dataset_transf, train_indexes) # transformed
#train_dataset_2 = Subset(train_dataset, train_indexes)
train_dataset = Subset(train_dataset, train_indexes)

In [0]:
#DATA AUGMENTATION -> we concatenate the original dataset with the transformed one
#sets = [train_dataset_1, train_dataset_2]
#train_dataset = ConcatDataset(sets)

In [10]:
len(train_dataset)

2892

**Prepare Dataloaders**

In [0]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True)

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

**Prepare Network**

In [12]:
if network == "alex":
  net = alexnet(pretrained = transferLearning) # Loading AlexNet model
  # if transferLearning = True, we use the pretrained net

  # AlexNet has 1000 output neurons, corresponding to the 1000 ImageNet's classes
  # We need 101 outputs for Caltech-101
  net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
                                                  # The convolutional layer is nn.Conv2d

# We just changed the last layer of AlexNet with a new fully connected layer with 101 outputs
# It is strongly suggested to study torchvision.models.alexnet source code

if network == "vgg":
  net = vgg16(pretrained="imagenet")
  net.classifier[6] = nn.Linear(4096, NUM_CLASSES)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


**Prepare Training**

In [0]:
# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# (nn.Module objects, like AlexNet, implement the Composite Pattern)
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 

if train_freeze == "all":
  parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet
elif train_freeze == "fc":
  parameters_to_optimize = net.classifier.parameters()
elif train_freeze == "cl":
  parameters_to_optimize = net.features.parameters()

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

**Training and Validation**

In [14]:
# By default, everything is loaded to cpu
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark # Calling this optimizes runtime

current_step = 0

#losses = []
#counterLosses = 0

accuracy_train = []
accuracy_val = []
loss_train = []
loss_val = []

#best_accuracy = 0
best_loss = 1000
best_epoch = 0
best_net = 0
acc = 0

# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  counterLosses = 0
  print()
  print('\nStarting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
  running_corrects = 0

  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() # Sets module in training mode

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network
    outputs_train = net(images)

    _, preds = torch.max(outputs_train.data, 1)

    running_corrects += torch.sum(preds == labels.data).data.item()

    # Compute loss based on output and ground truth
    loss = criterion(outputs_train, labels)

    #counterLosses += loss.item()*images.size(0)

    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1

  loss_train.append(loss.item())
  accuracy = running_corrects/float(len(train_dataset))
  accuracy_train.append(accuracy)

  net.train(False)

  running_correct_val = 0
  for images, labels in tqdm(val_dataloader):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    outputs_val = net(images)

    _, preds = torch.max(outputs_val.data, 1)

    running_correct_val += torch.sum(preds == labels.data).data.item()

    loss = criterion(outputs_val, labels)

  loss_val.append(loss.item())
  accuracy = running_correct_val/ float(len(val_dataset))
  accuracy_val.append(accuracy)

  """
  if accuracy > best_accuracy:
    best_accuracy = accuracy
    best_epoch = epoch
    best_net = copy.deepcopy(net.state_dict())
  """

  if loss.item() < best_loss:
    best_loss = loss.item()
    acc = accuracy
    best_epoch = epoch
    best_net = copy.deepcopy(net.state_dict())

  # Step the scheduler
  scheduler.step()

  #losses.append(counterLosses/float(len(train_dataset)))

print(f"\nThe best epoch is {best_epoch+1} with loss = {best_loss} and accuracy = {acc}")




Starting epoch 1/10, LR = [0.001]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Step 0, Loss 4.6649932861328125
Step 10, Loss 4.249819755554199
Step 20, Loss 2.9293761253356934
Step 30, Loss 3.2396597862243652
Step 40, Loss 2.745976209640503
Step 50, Loss 1.7849349975585938
Step 60, Loss 1.6708509922027588
Step 70, Loss 1.8259751796722412
Step 80, Loss 1.2945725917816162
Step 90, Loss 1.527274250984192
Step 100, Loss 1.4919296503067017
Step 110, Loss 0.8681522011756897
Step 120, Loss 0.7028505802154541
Step 130, Loss 0.8059684634208679
Step 140, Loss 1.1641367673873901
Step 150, Loss 0.4521486461162567
Step 160, Loss 1.0729159116744995
Step 170, Loss 0.8568102121353149


100%|██████████| 180/180 [00:21<00:00,  8.27it/s]



Starting epoch 2/10, LR = [0.001]


...

100%|██████████| 180/180 [00:21<00:00,  8.26it/s]



Starting epoch 10/10, LR = [0.001]


Step 1620, Loss 0.009793341159820557
Step 1630, Loss 0.0003210306167602539
Step 1640, Loss 0.0308687686920166
Step 1650, Loss 0.0030673742294311523
Step 1660, Loss 0.0007050037384033203
Step 1670, Loss 0.0016117095947265625
Step 1680, Loss 0.011597812175750732
Step 1690, Loss 0.00010269880294799805
Step 1700, Loss 5.21540641784668e-05
Step 1710, Loss 0.14564916491508484
Step 1720, Loss 0.0005182027816772461
Step 1730, Loss 5.072355270385742e-05
Step 1740, Loss 0.0013156533241271973
Step 1750, Loss 0.005546450614929199
Step 1760, Loss 0.003642439842224121
Step 1770, Loss 0.0005130767822265625
Step 1780, Loss 0.0005648136138916016
Step 1790, Loss 0.0001811981201171875


100%|██████████| 180/180 [00:22<00:00,  8.18it/s]


The best epoch is 8 with loss = 0.03835397958755493 and accuracy = 0.9014522821576764


In [0]:
best_net

In [0]:
# PLOT LOSSES
plt.plot(loss_train, label="Training")
plt.plot(loss_val, "r", label="Validation")
plt.legend(loc="upper right")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation losses")
plt.grid(axis="y")
plt.show()

In [16]:
loss_val

[0.5640168786048889,
 0.11916041374206543,
 0.37789058685302734,
 0.11801713705062866,
 0.30325281620025635,
 0.12105575203895569,
 0.09973520040512085,
 0.03835397958755493,
 0.3649991750717163,
 0.03836846351623535]

In [0]:
# PLOT ACCURACIES
plt.plot(accuracy_train, label="Training")
plt.plot(accuracy_val, "r", label="Validation")
plt.legend(loc="lower right")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training and Validation accuracies")
plt.grid(axis="y")
plt.show()

**Test**

In [18]:
net.load_state_dict(best_net)
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('\nTest Accuracy = {}'.format(accuracy))

100%|██████████| 181/181 [00:21<00:00,  8.48it/s]


Test Accuracy = 0.9101278949187694
